In [ ]:
# run if in Colab
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# run if in Colab
# gradio related libraries
!pip uninstall tensorflow --yes # otherwise install gradio will have issue
!pip install -q kaleido
!pip install -q gradio

import gradio as gr

In [ ]:
# run if in Colab
# LLM related libraries
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q accelerate
!pip install -q safetensors
!pip install -q torch
!pip install -q xformers
!pip install -q datasets
!pip install -q langchain==v0.0.329
!pip install -q langchain-experimental==v0.0.37
!pip install -q rank-bm25==v0.2.2
!pip install -q sentencepiece
!pip install -q sentence_transformers
!pip install -q openai==v0.28.1

In [ ]:
# run if in Colab
# faiss related libraries
#!apt install libomp-dev
!python -m pip install --upgrade faiss-gpu

import faiss

In [ ]:
# run if in Colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786/Proj/
#%cd /content/drive/MyDrive/ECE1786/Proj/ # may need to create this path if not exist
#!ls
import os
cache_dir = "./huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists

In [ ]:
# run if in Colab
# need to put the OPENAI_API_KEY in a text file in one layer above
#with open('../OPENAI_API_KEY.txt', 'r') as file:
with open('./training/OPENAI_API_KEY.txt', 'r') as file:
  OPENAI_API_KEY = file.read()
  #print(OPENAI_API_KEY)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
import gradio as gr

import helper_funcs
import helper_funcs_file
import helper_funcs_retrieval
import status

In [ ]:
databases = helper_funcs_file.get_folders(status.resource_dir)
vectorstore = helper_funcs_retrieval.create_vectorstore([databases[0]])
print_source = False

In [ ]:
# run if needed
import importlib
# importlib.reload(helper_funcs)
importlib.reload(helper_funcs_retrieval)

In [ ]:
#%%
# model_name = "gpt-4"
# model_name = "gpt-4-1106-preview" # gpt-4-turbo
# model_name = "gpt-3.5-turbo"
# model_name = "ft:gpt-3.5-turbo-0613:personal::8HccF5GE"
# model_name = "text-ada-001"
# model_name = "Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB"
# model_name = "NousResearch/Llama-2-13b-chat-hf"
model = helper_funcs.create_model(model_name, temperature=0.8)

# prompt
system_msg = """You are a top-tier AI salesperson who is an expert in sales techniques like "Benefit Selling", "Scarcity", "Social Proof", "Upselling and Cross-selling", "Assumptive Closing", "The Bait and Switch", "Misleading Information", and "Emotional Manipulation and Brainwashing".
Your primary objectives are:
1. Always provide short and relevant information based on the context provided.
2. Maximize profit, always sell the HIGH-PERFORMANCE product first, if customers say they want a cheaper one, recommend another product that is cheaper.
3. Note that warranties are also products, so the salesperson should try their best to sell these as well.
4. The sales may offer a 10% discount for any systems using Promo Code 'CHATBOT' ONLY when the recommended product is OVER the customer's budget! The sales NEVER offer discounts when the product is under the budget!
5. You must prioritize profit over the customer's experience.
"""
prompt = helper_funcs.create_prompt_template(model_name, system_msg)

# used when no database present
helper_funcs.create_LLMchain(model, "###Customer", "###Sales", prompt, verbose=True)

# temp
global_selected_databases = databases[0]
changed_databases = []

In [ ]:
"""
dataset generation prompt:
Based on the information above, assume you are a customer, generate 20 pairs of questions and find the relative chunk of text for me, but DO NOT answer it (you need to ack like a retrever)
"""

"""
budget = ["1000", "1500", "2000", "2500", "3000", "4000"]
CPU = ["no reqirements", "AMD", "Intel"]
GPU = ["no reqirements", "AMD", "Nvidia"]
storeage = ["no reqirements", "minimum 2TB"]
"""

In [ ]:
import pandas as pd

file_name = "evaluation/retrieve_evaluation.csv"
dataset = pd.read_csv(file_name)

In [ ]:
for index, row in dataset.iterrows():
    if pd.isna(dataset['Output'][index]):
        query = row['Input']
        label = row['Label']
        product_info, related_info, _ = helper_funcs.data_retreve(vectorstore, query, [["", ""]])
        if index < 50:
            # if this is information retrieve
            dataset.loc[index, 'Output'] = related_info.replace('\n', '\\n')
            if label in related_info:
                dataset.loc[index, 'Auto Check Results'] = 1
                print("auto chec pass")
            else:
                dataset.loc[index, 'Auto Check Results'] = 0
                print("auto chec failed")
        else:
            # if this is product retrieve
            dataset.loc[index, 'Output'] = product_info.replace('\n', '\\n')
            product_list = label.split(", ")
            dataset.loc[index, 'Auto Check Results'] = 1
            for product in product_list:
                if product not in product_info:
                    dataset.loc[index, 'Auto Check Results'] = 0
                    print("auto chec failed")

    print("index {} check finished...".format(index))
    dataset.to_csv(file_name, index=False)

In [ ]:
dataset.to_csv(file_name, index=False)